In [42]:
import torch
import pickle
from pathlib import Path
from recsys_slates_dataset import dataset_torch
import numpy as np
import os
import json
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
import torch
import fasttext.util
import fasttext


In [43]:
# fasttext.util.download_model('en', if_exists='ignore') 
ft = fasttext.load_model('cc.en.300.bin')
fasttext.util.reduce_model(ft, 100)
ft.get_dimension()

100

In [44]:
output_file = "/home/aayush/rsys_data/rsys_2025/item_embeddings_finn.pkl"

with open(output_file, 'rb') as f:
    item_embedding_dict = pickle.load(f)


print(f"Loaded {len(item_embedding_dict)} item embeddings.")
print(f"Embedding for item 1: {item_embedding_dict[2]}")

Loaded 1311774 item embeddings.
Embedding for item 1: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [45]:

current_directory = os.getcwd() 
json_path = os.path.join(current_directory, "finn_data", "ind2val.json")

with open(json_path, "r") as file:
    ind2val = json.load(file)

for key, val in ind2val.items():
  print(" ")
  print(f"{key} first entries:")
  for idx, name in val.items():
    print(f"{idx}: {val[idx]}")
    

# Explore the keys and values
print("Keys in ind2val.json:", ind2val.keys())
print("Category Mapping Sample:", list(ind2val["category"].items())[:-3])
print("Interaction Type Mapping Sample:", list(ind2val["interaction_type"].items()))


 
category first entries:
0: PAD
1: noClick
2: <UNK>
3: BAP,antiques,Trøndelag
4: MOTOR,,Sogn og Fjordane
5: BAP,electronicsappliances,Finnmark
6: BAP,entertainmenthobbyleisure,Østfold
7: MOTOR,,Hedmark
8: BAP,antiques
9: BOAT,,Møre og Romsdal
10: BAP,animalsequipment,Østfold
11: BAP,entertainmenthobbyleisure,Møre og Romsdal
12: BAP,housegardenrenovation,Østfold
13: REAL_ESTATE,,Møre og Romsdal
14: BAP,business,Oppland
15: BAP
16: BAP,housegardenrenovation,Rogaland
17: BAP,housegardenrenovation,Vestfold
18: BAP,business,Troms
19: BAP,electronicsappliances,
20: BAP,furnitureinterior,Møre og Romsdal
21: BAP,parentschildren,Møre og Romsdal
22: BAP,sportsoutdoors,Vest-Agder
23: BAP,animalsequipment,Sogn og Fjordane
24: BAP,electronicsappliances,Sogn og Fjordane
25: BAP,sportsoutdoors,Nordland
26: BAP,sportsoutdoors,Rogaland
27: MOTOR,,Møre og Romsdal
28: BAP,animalsequipment,Rogaland
29: BAP,electronicsappliances,Trøndelag
30: BAP,sportsoutdoors,Buskerud
31: BAP,parentschildren,Trøndelag
3

In [46]:
itemattr_path = os.path.join(current_directory, "finn_data", "itemattr.npz")
itemattr_data = np.load(itemattr_path, allow_pickle=True)
category=itemattr_data['category']
print("Category shape:", category.shape)


Category shape: (1311775,)


In [55]:

embedding_dict = {}

# Iterate over each index and category in ind2val
for idx, categories in ind2val['category'].items():
    # Concatenate category values (you can adjust this as needed)
 
    # category_text = ', '.join(categories.values()) 
    
    # Get the embedding for the category text

    embedding = ft.get_sentence_vector(categories)  # Get the sentence-level embedding
    
    # Store the embedding in the dictionary with the index as the key
    embedding_dict[int(idx)] = embedding

# Now `embedding_dict` will contain the embeddings for each index
print("Generated embeddings:", embedding_dict)


Generated embeddings: {0: array([-2.57718861e-01, -1.05258957e-01,  3.18845809e-01,  1.44435763e-01,
        4.17068377e-02,  4.28794883e-02,  1.25232503e-01,  1.07191369e-01,
        1.40067101e-01,  6.59290627e-02,  3.80559862e-02, -9.60575789e-03,
       -6.90506920e-02, -1.39320120e-01,  5.05953915e-02,  2.09249303e-01,
        1.13005666e-02, -2.59869192e-02, -1.13222703e-01, -2.02568993e-02,
       -1.93106204e-01,  6.50415197e-03, -1.06598459e-01,  1.20764794e-02,
        1.52674019e-01,  3.28168780e-01,  3.80235426e-02,  1.29127383e-01,
        8.00623372e-02, -5.56052886e-02,  3.02339606e-02,  5.83269782e-02,
        1.97923079e-01,  3.38300876e-02, -6.41415268e-02, -3.20009440e-02,
        2.01193267e-04, -7.10527524e-02, -1.52667448e-01,  5.36527969e-02,
        5.95771112e-02, -3.15886289e-02, -3.40322182e-02, -1.17234625e-01,
        1.12761684e-01,  3.66179906e-02, -4.23324667e-02,  4.92567904e-02,
       -6.09737858e-02, -3.22863758e-02,  6.53305603e-03, -1.28103763e-01,

In [61]:
base_path = Path("/home/aayush/rsys_data/rsys_2025")

# Ensure the directory exists
base_path.mkdir(parents=True, exist_ok=True)

# Define the path for the pickle file
pickle_path = base_path / "category_embeddings_fin.pkl"

# Save the dictionary as a pickle file
with open(pickle_path, "wb") as f:
    pickle.dump(embedding_dict, f)

print(f"Embeddings saved to {pickle_path}")

Embeddings saved to /home/aayush/rsys_data/rsys_2025/category_embeddings_fin.pkl


In [62]:
with open(pickle_path, "rb") as f:
    loaded_embedding_dict = pickle.load(f)

# Print the loaded dictionary to cross-verify
print("Loaded embeddings:", loaded_embedding_dict)

Loaded embeddings: {0: array([-2.57718861e-01, -1.05258957e-01,  3.18845809e-01,  1.44435763e-01,
        4.17068377e-02,  4.28794883e-02,  1.25232503e-01,  1.07191369e-01,
        1.40067101e-01,  6.59290627e-02,  3.80559862e-02, -9.60575789e-03,
       -6.90506920e-02, -1.39320120e-01,  5.05953915e-02,  2.09249303e-01,
        1.13005666e-02, -2.59869192e-02, -1.13222703e-01, -2.02568993e-02,
       -1.93106204e-01,  6.50415197e-03, -1.06598459e-01,  1.20764794e-02,
        1.52674019e-01,  3.28168780e-01,  3.80235426e-02,  1.29127383e-01,
        8.00623372e-02, -5.56052886e-02,  3.02339606e-02,  5.83269782e-02,
        1.97923079e-01,  3.38300876e-02, -6.41415268e-02, -3.20009440e-02,
        2.01193267e-04, -7.10527524e-02, -1.52667448e-01,  5.36527969e-02,
        5.95771112e-02, -3.15886289e-02, -3.40322182e-02, -1.17234625e-01,
        1.12761684e-01,  3.66179906e-02, -4.23324667e-02,  4.92567904e-02,
       -6.09737858e-02, -3.22863758e-02,  6.53305603e-03, -1.28103763e-01,
  